# Analyzing SQL Workloads

This Jupyter Notebook contains examples of how to use the Open APIs to run SQL and use Visual Exaplain in the Db2 Data Management Console. 

### Import Helper Classes
For more information on these classes, see the Lab on Automate Db2 with Open Console Services

In [1]:
%run ./dmc_setup.ipynb

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [2]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
profile  = 'SAMPLE'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
profileURL = "?profile="+profile
databaseAPI = Db2(Console+'/dbapi/v4')

if databaseAPI.authenticate(user, password, profile) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

Token Created


### Confirm the connection
To confirm that your connection is working you can check the status of the moitoring service. You can also check your console connection to get the details of the specific database connection you are working with. Since your console user id and password are may be limited as to which databases they can access you need to provide the connection profile name to drill down on any detailed information for the database.

In [3]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

{'name': 'SAMPLE', 'disableDataCollection': 'false', 'databaseVersion': '11.5.0', 'databaseName': 'SAMPLE', 'timeZone': '-50000', 'DB2Instance': 'db2inst1', 'db2license': 'AESE,DEC', 'isInstPureScale': 'false', 'databaseVersion_VRMF': '11.5.0.0', 'sslConnection': 'false', 'userProfileRole': 'OWNER', 'timeZoneDiff': '0', 'host': 'localhost', '_PROFILE_INIT_': 'true', 'dataServerType': 'DB2LUW', 'port': '50000', 'URL': 'jdbc:db2://localhost:50000/SAMPLE', 'edition': 'AESE,DEC', 'isInstPartitionable': 'false', 'dataServerExternalType': 'DB2LUW', 'capabilities': '["DSM_ENTERPRISE_LUW"]', 'OSType': 'Linux', 'location': '', 'tag': ['HOT']}


### Using the Db2 class to run SQL
You can use the console API to run single SQL statement or sets of statements in a single call. A step by step explanation of running SQL through the Db2 Data Management Console SQL service is available in the [Db2 SQL with RESTful Services](http://localhost:8888/notebooks/Db2_RESTful_APIS.ipynb) Jupyter notebook. These examples use the Db2 class defined in [dmc_setup.ipynb](http://localhost:8888/notebooks/dmc_setup.ipynb). The Db2 class takes care of creating a reusable authentication key from your login information. It includes two routines **runSQL** and **getSQLJobResult** that make submitting SQL scripts easy. **runSQL** uses the console service to run one or more statements in the background, saving the result. **getSQLJobResult** lets you access the results of each statement.  

In this first example we will run a single statement. The first step defines a single statement, runs the SQL and retrieves a unique id to identify the SQL running in the background. Using that id you can check the job results to see if it is still running or complete. 

In [4]:
#Run SQL Statement and Retrieve the identifier used to track the run

# Define the SQL to Run
sqlText = 'select TABSCHEMA, TABNAME from syscat.tables'

# Run SQL throught the Console SQL Editor REST Service
r = databaseAPI.runSQL(sqlText)

# Retrieve the run identifier from the SQL Editor Service
runID = databaseAPI.getJSON(r)['id'] 

# Using the saved runID retrieve a JSON description of the run
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID)) 

# Extract the status of the run from the JSON return and print the runID as well as its status
print(runID+" "+json['status']) 

1576017452343_726745000 running


In the next step we check that the statement has finished running. Then we check for three possible conditions. First,there was an error in the SQL Statement. If so we print the error message. Second, the statement ran successfully and there is a result set. If so we print out the results as a table. And third, there were no errors but also no results were returned. Try running different SQL Statements to see the different results. 

In [5]:
# ONLY RUN THIS STEP IF THE STATEMENT DID NOT COMPLETE IN THE PREVIOUS STEP
# Repeat this step until you see that the statement completed
# Once you have retrieved the results you cannot run the same call a second time successfully.
# The JSON structure is only available on the first call.
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
print(json)

{'id': '1576017452343_726745000', 'results': [{'exceed_size': 'yes', 'rows_count': 11, 'runtime_seconds': 0.013000000268220901, 'columns': ['TABSCHEMA', 'TABNAME'], 'columns_type': ['string', 'string'], 'limit': 10, 'index': 0, 'rows': [['DB2INST1', 'ACT'], ['DB2INST1', 'AS_EMP'], ['DB2INST1', 'BASE_EMP_TXS'], ['DB2INST1', 'CENTRAL_LINE'], ['DB2INST1', 'CL_SCHED'], ['DB2INST1', 'CUSTOMERS'], ['DB2INST1', 'DEMO_TABLE'], ['DB2INST1', 'DEPARTMENT'], ['DB2INST1', 'DEPARTMENTS'], ['DB2INST1', 'DEPT']], 'command': 'select TABSCHEMA, TABNAME from syscat.tables'}], 'status': 'completed'}


Now let us unpack the JSON that was returned above. The code below checks that the statement completed and the checks for errors. If there are no errors and rows were returned in a result set then the rows are extracted to a dataframe and displayed. Dataframes are a very powerful part of the Python Panda's library. They let us easily manupulate data sets that come back from Db2. 

In [6]:
if json['results'] != []:
        results = json['results'][0]
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. No results')

  TABSCHEMA       TABNAME
0  DB2INST1           ACT
1  DB2INST1        AS_EMP
2  DB2INST1  BASE_EMP_TXS
3  DB2INST1  CENTRAL_LINE
4  DB2INST1      CL_SCHED
5  DB2INST1     CUSTOMERS
6  DB2INST1    DEMO_TABLE
7  DB2INST1    DEPARTMENT
8  DB2INST1   DEPARTMENTS
9  DB2INST1          DEPT


You do not want to have to continously check to see if a statement is complete. So let us use a simple loop to do the checking for us. 

In [7]:
# Run SQL Statement and Retrieve the identifier used to track the run
sqlText = 'select TABSCHEMA, TABNAME from syscat.tables'
runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 
print('Run Identifier: '+str(runID))

# Check to see if the statement finished running
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

# If the statement still has not finished wait in one second intervals
# we check for results because depending on timing it is possible to retrieve results without the status being complete.
# Once we retrieve the results there are removed from the service. 
while json['results'] == [] :
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
    time.sleep(1) 
    
# Assuming we only have one statement, unpack the results    
results = json['results'][0]
if 'error' in results : 
    print(results['error'])
elif 'rows' in results :
    df = pd.DataFrame(results['rows'],columns=results['columns'])
    print(df)
else :
    print('No errors. No results')

Run Identifier: 1576017459970_118463755
  TABSCHEMA       TABNAME
0  DB2INST1           ACT
1  DB2INST1        AS_EMP
2  DB2INST1  BASE_EMP_TXS
3  DB2INST1  CENTRAL_LINE
4  DB2INST1      CL_SCHED
5  DB2INST1     CUSTOMERS
6  DB2INST1    DEMO_TABLE
7  DB2INST1    DEPARTMENT
8  DB2INST1   DEPARTMENTS
9  DB2INST1          DEPT


### SQL Scripts Used in the Lab
We are going to define a few scripts that we will use during this lab. The first two will define tables that are used during the rest of the lab. The third defines a workload we will reuse. Notice that these scripts contain multiple SQL statements. We are going to have to update our code to handle that.

In [8]:
sqlScriptCreateEmployee = \
'''
CREATE TABLE IF NOT EXISTS EMPLOYEES (ENO INTEGER, DEPTNO INTEGER, LASTNAME VARCHAR(30),
    HIREDATA DATE, SALARY INTEGER);

INSERT INTO EMPLOYEES
-- generate 500000 records
    WITH DT(ENO) AS (VALUES(1) UNION ALL SELECT ENO+1 FROM DT WHERE ENO < 500000)

-- Now, use the generated records in DT to create other columns
-- of the employee record.
    SELECT ENO,
    RAND() * 500,
    TRANSLATE(CHAR(INTEGER(RAND()+500000)),
    CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
        WHEN 1 THEN 'aeiou' || 'hjklm'
        WHEN 2 THEN 'aeiou' || 'npqrs'
        WHEN 3 THEN 'fredr' || 'annab'
        ELSE 'aeiou' || 'twxyz' END,
        '1234567890') AS LASTNAME,
   CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
   INTEGER(1000*RAND()*200) AS SALARY
   FROM DT;

SELECT COUNT FROM EMPLOYEES;

SELECT * FROM EMPLOYEES ORDER BY ENO;
'''

In [9]:
sqlScriptCreateDepartment = \
'''
CREATE TABLE IF NOT EXISTS DEPARTMENTS (DEPTNO INTEGER, DEPTNAME VARCHAR(30), REVENUE INTEGER);

INSERT INTO DEPARTMENTS
-- generate 500 department records
WITH DT(DEPTNO) AS (VALUES(1) UNION ALL SELECT DEPTNO+1 FROM DT WHERE DEPTNO < 500 )

    SELECT DEPTNO,
        TRANSLATE(CHAR(INTEGER(RAND()+500000)),
            CASE MOD(DEPTNO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
                WHEN 1 THEN 'aeiou' || 'hjklm'
                WHEN 2 THEN 'aeiou' || 'npqrs'
                WHEN 3 THEN 'fredr' || 'annab'
                ELSE 'aeiou' || 'twxyz' END,
                '1234567890') AS DEPTNAME,
        INTEGER(10000*RAND()*50000) AS REVENUE
        FROM DT;

SELECT COUNT FROM DEPARTMENTS;

SELECT * FROM DEPARTMENTS;
'''

In [10]:
sqlScriptWorkload1 = \
'''
WITH SALARYBY (DEPTNO, TOTAL) AS
    (SELECT DEPT.DEPTNO DNO, SUM(BIGINT(EMP.SALARY)) TOTAL_SALARY
        FROM EMPLOYEES EMP, DEPARTMENTS DEPT
        WHERE DEPT.DEPTNO = EMP.DEPTNO AND EMP.SALARY > 190000
        GROUP BY DEPT.DEPTNO
        ORDER BY DNO)
SELECT DEPT.DEPTNAME NAME, SALARYBY.TOTAL COST, DEPT.REVENUE, DEPT.REVENUE-SALARYBY.TOTAL PROFIT
FROM SALARYBY, DEPARTMENTS DEPT
WHERE DEPT.DEPTNO = SALARYBY.DEPTNO
AND REVENUE > TOTAL
ORDER BY PROFIT
'''

In [11]:
sqlScriptWorkload2 = \
'''
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO AND EMP.SALARY < 50000
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY
  FROM EMPLOYEES EMP, DEPARTMENTS DEPT 
  WHERE DEPT.DEPTNO = EMP.DEPTNO AND EMP.SALARY < 190000
  GROUP BY DEPT.DEPTNO
  ORDER BY DNO;

SELECT DEPT.DEPTNO, DEPT.REVENUE
  FROM DEPARTMENTS DEPT WHERE DEPT.REVENUE > 450000000;
'''

### Running Multiple SQL Statements in a Single Script
In the following example we run multiple statements in a single API call. When the statement is started we get a run handle we can use to access the results of the SQL execution. Running the script works the same as running a single statement. However, retrieving results is more complex. Each time you check for results you may get the results of one or more statements. Like before, you can only retrieve the results for each statement once. 

In [12]:
# Run SQL Statement and Retrieve the identifier used to track the run

sqlText = sqlScriptCreateEmployee
runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 
print(runID) 

1576017468460_1369547729


In [13]:
# We need to create a loop to check for results 
# Each time we retrieve new results they are added to the fulljson result set

json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
fulljson = json

while json['results'] != [] or json['status'] != "completed" :
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
    for results in json['results'] :
        fulljson['results'].append(results)
    time.sleep(1) 

In [14]:
# We need to unpack the entire JSON result set
# and display if each statement ran successfully and how long it took
# we also extract how many rows were affected for insert statements

for results in fulljson['results']:
    print('Statement: '+str(results['index'])+': '+results['command'])
    print('Runtime ms: '+str(results['runtime_seconds']*1000))
    if 'error' in results : 
        print(results['error'])
    elif 'rows' in results :
        df = pd.DataFrame(results['rows'],columns=results['columns'])
        print(df)
    else :
        print('No errors. Row Affected: '+str(results['rows_affected']))
    print()
    print('* * * * * * * * * * * * * * *')

Statement: 0: CREATE TABLE IF NOT EXISTS EMPLOYEES (ENO INTEGER, DEPTNO INTEGER, LASTNAME VARCHAR(30),
    HIREDATA DATE, SALARY INTEGER)
Runtime ms: 19.999999552965164
No errors. Row Affected: 0

* * * * * * * * * * * * * * *
Statement: 1: INSERT INTO EMPLOYEES
-- generate 500000 records
    WITH DT(ENO) AS (VALUES(1) UNION ALL SELECT ENO+1 FROM DT WHERE ENO < 500000)

-- Now, use the generated records in DT to create other columns
-- of the employee record.
    SELECT ENO,
    RAND() * 500,
    TRANSLATE(CHAR(INTEGER(RAND()+500000)),
    CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
        WHEN 1 THEN 'aeiou' || 'hjklm'
        WHEN 2 THEN 'aeiou' || 'npqrs'
        WHEN 3 THEN 'fredr' || 'annab'
        ELSE 'aeiou' || 'twxyz' END,
        '1234567890') AS LASTNAME,
   CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
   INTEGER(1000*RAND()*200) AS SALARY
   FROM DT
Runtime ms: 1611.9999885559082
No errors. Row Affected: 500000

* * * * * * * * * * * * * * *
Statement: 2: SELECT COU

### Creating a Routine to Run an SQL Script
To make things easier we can create reusable routines that will included everything we have developed so far. By running the next two steps, you create two routines that you can call by passing parameters to them. 

While we could create a single routine to run SQL and then display the results, we are creating two different routines so that we can display the results differently later in the lab. 

In [20]:
def runSQL(profile,user, password, sqlText):
    
    if databaseAPI.authenticate(user, password, profile) :

        runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        fulljson = json

        while json['results'] == [] or json['status'] != "completed" :
            json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
            for results in json['results'] :
                fulljson['results'].append(results)
            time.sleep(1) 
        return fulljson
    else :
        print('Could not authenticate')

In [21]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        print('Runtime ms: '+str(results['runtime_seconds']*1000))
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()

Now call the new routine using the parameters to define the connection profile to use, the userid and password and the SQL Text. In this example we will drop the table we just created. Try running it a second time. You can see that it correctly returns an error message. 

In [17]:
profile = 'SAMPLE'
user = 'DB2INST1'
password = 'db2inst1'
sqlText = 'DROP TABLE EMPLOYEES'
displayResults(runSQL(profile, user, password, sqlText))

Statement: 0: DROP TABLE EMPLOYEES
Runtime ms: 27.000000700354576
No errors. Row Affected: 0



### Running Multiple Scripts across Multiple Databases
Now we will use our new routines to create the Employees and the Department tables across two databases. In this lab we can use the SAMPLE and the HISTOROY database. The example below loops through two databases and two scripts to create the required tables.

In [22]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptCreateEmployee, sqlScriptCreateDepartment]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print('Running Scripts against profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

* * * * * * * * * * * * * * * * * * * * * * * * * * *
Running Scripts against profile: SAMPLE
Statement: 0: CREATE TABLE IF NOT EXISTS EMPLOYEES (ENO INTEGER, DEPTNO INTEGER, LASTNAME VARCHAR(30),
    HIREDATA DATE, SALARY INTEGER)
Runtime ms: 14.000000432133675
No errors. Row Affected: 0

Statement: 1: INSERT INTO EMPLOYEES
-- generate 500000 records
    WITH DT(ENO) AS (VALUES(1) UNION ALL SELECT ENO+1 FROM DT WHERE ENO < 500000)

-- Now, use the generated records in DT to create other columns
-- of the employee record.
    SELECT ENO,
    RAND() * 500,
    TRANSLATE(CHAR(INTEGER(RAND()+500000)),
    CASE MOD(ENO,5) WHEN 0 THEN 'aeiou' || 'bcdfg'
        WHEN 1 THEN 'aeiou' || 'hjklm'
        WHEN 2 THEN 'aeiou' || 'npqrs'
        WHEN 3 THEN 'fredr' || 'annab'
        ELSE 'aeiou' || 'twxyz' END,
        '1234567890') AS LASTNAME,
   CURRENT DATE - (RAND()*10957) DAYS AS HIREDATE,
   INTEGER(1000*RAND()*200) AS SALARY
   FROM DT
Runtime ms: 1845.0000286102295
No errors. Row Affected

### Running multiple scripts across multiple databases - Summarized Results
Now that we have our tables created on both databases, we can run workloads and measure their performance. By repeatedly running the scripts across multiple databases in a single Db2 instance we can simulate a real database environemt. 

Instead of using the displayResults routine we are going to capture runtime metrics for each run of the SQL Query workloads so that we can analyze performance. The appendResults routine builds this dataframe with each pass.

runScripts lets use define the database connection profiles we want to run against, the scripts to run, and now many times to repeat the runs for each profile and for each script.

In [23]:
# This routine builds up a Data Frame containing the run results as we run workloads across databases
def appendResults(df, profile, json) :
    
    error = ''
    rows = 0
    for results in json['results']:
        if 'error' in results : 
            error = results['error']
        if 'rows_affected' in results : 
            rows = results['rows_affected']
        df = df.append({'profile':profile,'index':results['index'], 'statement':results['command'], 'error':error, 'rows_affected': rows, 'runtime_ms':(results['runtime_seconds']*1000)}, ignore_index=True)
    return df

In [24]:

def runScripts(profileList, scriptList, user, password, profileReps, scriptReps) :

    df = pd.DataFrame(columns=['profile', 'index', 'statement', 'error', 'rows_affected', 'runtime_ms'])
    
    for x in range(0, profileReps):
        for profile in profileList :
            for y in range(0, scriptReps) :
                for script in scriptList :
                    json = runSQL(profile, user, password, script)
                    df = appendResults(df, profile, json)
                        
    return df

In [25]:
# Retrieve the current package cache list 
# Show the first ten as sorted by the statement execution time
def getCurrentPackageCacheListDF(profile) :
    databaseAPI.authenticate(user, password, profile)
    r = databaseAPI.getCurrentPackageCacheList("false")
    if (databaseAPI.getStatusCode(r)==200):
        json = databaseAPI.getJSON(r)
        if json['count'] > 0:  
            df = pd.DataFrame(json_normalize(json['resources']))
            df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
            return df
        else: 
            print('No data returned')  
    else:
        print(databaseAPI.getStatusCode(r))

The next cell loops through a list of databases as well as a list of scripts and run then repeatedly.

We can then collect the most recent package cache information to get another view of performance information. 

**Note:** There is a delay while the Db2 Data Management Console collects this information. The following script may not find any rows immediately.

In [26]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptWorkload1, sqlScriptWorkload2]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 2
scriptReps = 2

df = runScripts(profileList, scriptList, user, password, profileReps, scriptReps)

display(df)

for profile in profileList:
    print('Profile: '+profile)
    packageCacheDF = getCurrentPackageCacheListDF(profile)
    display(packageCacheDF[['stmt_text','stmt_exec_time_ms','stmtid']])

,profile,index,statement,error,rows_affected,runtime_ms
0,SAMPLE,0,"WITH SALARYBY (DEPTNO, TOTAL) AS\n (SELECT ...",,0,416.999996
1,SAMPLE,0,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",,0,141.000003
2,SAMPLE,1,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",,0,61.999999
3,SAMPLE,2,"SELECT DEPT.DEPTNO, DEPT.REVENUE\n FROM DEPAR...",,0,3.000000
4,SAMPLE,0,"WITH SALARYBY (DEPTNO, TOTAL) AS\n (SELECT ...",,0,48.999999
5,SAMPLE,0,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",,0,78.000002
6,SAMPLE,1,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",,0,61.000001
7,SAMPLE,2,"SELECT DEPT.DEPTNO, DEPT.REVENUE\n FROM DEPAR...",,0,1.000000
8,HISTORY,0,"WITH SALARYBY (DEPTNO, TOTAL) AS\n (SELECT ...",,0,52.000001
9,HISTORY,0,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",,0,57.000000


Profile: SAMPLE


,stmt_text,stmt_exec_time_ms,stmtid
3,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",2947,-140115937412306713
0,"WITH SALARYBY (DEPTNO, TOTAL) AS (SELECT DEPT....",587,2635305026549059699
1,"SELECT DEPT.DEPTNO, DEPT.REVENUE FROM DEPARTME...",10,-6291622876202073758
2,"call SYSIBM.SQLCAMESSAGECCSID(?,?,?,?,?,?,?,?,...",0,8110287721056492873


Profile: HISTORY


,stmt_text,stmt_exec_time_ms,stmtid
3,"SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY))...",517,-140115937412306713
0,"WITH SALARYBY (DEPTNO, TOTAL) AS (SELECT DEPT....",191,2635305026549059699
1,"SELECT DEPT.DEPTNO, DEPT.REVENUE FROM DEPARTME...",0,-6291622876202073758
2,"call SYSIBM.SQLCAMESSAGECCSID(?,?,?,?,?,?,?,?,...",0,8110287721056492873


### Analyze Results
Now we can use the results in the dataframe to look at the results statistically. First we can see the average runtime for each statement across the databases.

In [27]:
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtMean = df.groupby(['statement']).mean()
print(stmtMean)

Mean runtime in ms
                                                                                                      runtime_ms
statement                                                                                                       
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY\n  FROM EMPLOYEES EMP, DEPARTMENTS DE...   70.125000
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY\n  FROM EMPLOYEES EMP, DEPARTMENTS DE...   73.500001
SELECT DEPT.DEPTNO, DEPT.REVENUE\n  FROM DEPARTMENTS DEPT WHERE DEPT.REVENUE > 450000000                1.750000
WITH SALARYBY (DEPTNO, TOTAL) AS\n    (SELECT DEPT.DEPTNO DNO, SUM(BIGINT(EMP.SALARY)) TOTAL_SALA...   96.750000


We can also display the total runtime for each statement across databases.

In [28]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

Total runtime in ms
                                                                                                      runtime_ms
statement                                                                                                       
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY\n  FROM EMPLOYEES EMP, DEPARTMENTS DE...  561.000001
SELECT DEPT.DEPTNO DNO, SUM(FLOAT(EMP.SALARY)) TOTAL_SALARY\n  FROM EMPLOYEES EMP, DEPARTMENTS DE...  588.000011
SELECT DEPT.DEPTNO, DEPT.REVENUE\n  FROM DEPARTMENTS DEPT WHERE DEPT.REVENUE > 450000000               14.000000
WITH SALARYBY (DEPTNO, TOTAL) AS\n    (SELECT DEPT.DEPTNO DNO, SUM(BIGINT(EMP.SALARY)) TOTAL_SALA...  773.999996


We can even graph the total run time for all the statements can compare database performance. Since the History database is actively being used to collect monitoring data it is usually the slower of the two. 

In [ ]:
import matplotlib.pyplot as plt
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
profileSum = df.groupby(['profile']).sum()
profileSum.plot(kind='bar')
plt.show()

You can also use the Db2 Data Management Console directly to look at the package cache and the package cache history.

Run the following cell and log in using **DB2INST1** as the userid and password. The cell calls the console as a micro-service and includes part of the live user interface into a IFrame directly in this notebook.

Change the timeframe from **Latest** to **Last 1 hour** so you can see a history of package cache monitoring.

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/package_cache'+profileURL, width=1400, height=480)

## Analyze a Single Statement
You can identify the slowest SQL statement and the slowest database to focus your attention. The next two cells will to that by scanning through our previous results.

In [ ]:
print('Slowest SQL Statement')
slowestSQL = stmtSum['runtime_ms'].idxmax()
print(slowestSQL)

In [ ]:
print('Slowest Database')
slowestProfile = profileSum['runtime_ms'].idxmax()
print(slowestProfile)

A very powerful tool that you can include directly into your notebook is Visual Explain. Just like being able to explore the package cache history you can call the micro-service that provides a live intertive interface. In this example we will take the slowest statment on the slowest database and explain the statement. 

In [ ]:
# Visually explain the access plan for an SQL Statement
profileURL = "?profile="+slowestProfile
stmt = slowestSQL.replace('\n', ' ').replace('\r', '')
IFrame(database+'/console/?mode=compact#sql/explain/create/'+stmt +profileURL, width=1400, height=480)

### Comparing Results
Now we can create indexes to see if we can improve the overall performance of the workloads. The next cell will create indexes on the SALARY column of the Employees table as well as the DEPTO of both the DEPARTMENTS and EMPLOYEES tables. 

In [ ]:
createIndexes = \
'''
CREATE INDEX ix_salary
ON EMPLOYEES(SALARY);

CREATE INDEX ix_empdept
ON EMPLOYEES(DEPTNO);

CREATE INDEX ix_dept
ON DEPARTMENTS(DEPTNO);
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [createIndexes]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

Lets see if visual explain predicts any improvements.

In [ ]:
# Visually explain the access plan for an SQL Statement
profileURL = "?profile="+slowestProfile
stmt = slowestSQL.replace('\n', ' ').replace('\r', '')
IFrame(database+'/console/?mode=compact#sql/explain/create/'+stmt +profileURL, width=1400, height=480)

Now we will repeat the original workload performance run to see if there is a measureable difference in performance. 

In [ ]:
profileList = ['SAMPLE','HISTORY']
scriptList = [sqlScriptWorkload1, sqlScriptWorkload2]
user = 'DB2INST1'
password = 'db2inst1'
profileReps = 2
scriptReps = 2

df = runScripts(profileList, scriptList, user, password, profileReps, scriptReps)

display(df)

for profile in profileList:
    print('Profile: '+profile)
    packageCacheDF = getCurrentPackageCacheListDF(profile)
    display(packageCacheDF[['stmt_text','stmt_exec_time_ms','stmtid']])

In [ ]:
print('Total runtime in ms')
pd.set_option('display.max_colwidth', 100)
stmtSum = df.groupby(['statement']).sum()
print(stmtSum)

Compare these results to your original performance without indexes.

In [ ]:
import matplotlib.pyplot as plt
print('Mean runtime in ms')
pd.set_option('display.max_colwidth', 100)
profileSum = df.groupby(['profile']).sum()
profileSum.plot(kind='bar')
plt.show()

### Clean Up

If you want to run the lab again, or try with different indexes, you can use the cells below to drop either the current indexes or the original tables. 

Experiment with different SQL Statements, and different tables to see what your results look like. You can even try to create new cells that will run different workloads on different databases. 

In [ ]:
dropIndexes = \
'''
DROP INDEX ix_salary;
DROP INDEX ix_empdept;
DROP INDEX ix_dept;
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [dropIndexes]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

In [19]:
dropTables = \
'''
DROP TABLE EMPLOYEES;
DROP TABLE DEPARTMENTS;
'''
profileList = ['SAMPLE','HISTORY']
scriptList = [dropTables]
user = 'DB2INST1'
password = 'db2inst1'

for profile in profileList :
    print('Profile: '+profile)
    for script in scriptList :
        json = runSQL(profile, user, password, script)
        displayResults(json)
print('done')

Profile: SAMPLE
Statement: 0: DROP TABLE EMPLOYEES
Runtime ms: 30.99999949336052
No errors. Row Affected: 0

Statement: 1: DROP TABLE DEPARTMENTS
Runtime ms: 10.999999940395355
No errors. Row Affected: 0

Profile: HISTORY
Statement: 0: DROP TABLE EMPLOYEES
Runtime ms: 16.00000075995922
No errors. Row Affected: 0

Statement: 1: DROP TABLE DEPARTMENTS
Runtime ms: 8.00000037997961
No errors. Row Affected: 0

done


#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]